<a href="https://colab.research.google.com/github/yexf308/AdvancedMachineLearning/blob/main/Homework/HW1/HW1Q5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pylab inline 
import numpy.linalg as LA
from time import time
from scipy.sparse.csgraph import dijkstra
import plotly.express as px
from scipy import sparse
import pandas as pd


Populating the interactive namespace from numpy and matplotlib


# Q5: SNPs of World-wide Populations (Group Homework) (60pts) (**)
### Your group member: 
For this homework, the dataset is very large(~1G) so we need to use the google drive to store the data. You need to allow the colab access your google drive. If you feel unsafe to your own personal data, you can create a new google account, simply for the purpose of data analysis. 


**Data description:**
This is the SNPs (Single Nucleid Polymorphisms) dataset in
Human Genome Diversity Project (HGDP). Refer to the latest science paper: https://www.science.org/doi/10.1126/science.aay5012. 

- Data matrix of $N\times D$ for $N=1043$ individuals around the world and $D=488919$ columns of SNPs. 

- Each entry in the matrix has 0, 1, 2, representing genotype AA, AC, CC. 


**Data access:** the SNP data is stored in https://drive.google.com/file/d/1a9I8_akfCMHBRrPMdnWkjyL9fKcQbJJq/view?usp=sharing. 

**Step1**: Once you open the link in the brower, make sure you click the "Add shortcut to Drive" and now your google drive should show up the zip file.  Then you run the following code to link colab to your google drive. Data credit to Yuan Yao, Quanhua MU and Yoonhee Nam.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os 
os.chdir("/content/drive/My Drive")

Mounted at /content/drive


**Step2**: The name of the file is `ceph_hgdp_minor_code_XNA.betterAnnotated.csv.zip` so we need to unzip it. The following code is to unzip this file. You only need to run this code once. If your google drive has the file `ceph_hgdp_minor_code_XNA.betterAnnotated.csv`, you can skip this line.   

In [ ]:
!unzip ceph_hgdp_minor_code_XNA.betterAnnotated.csv.zip

**Step3**: Here we are using pandas to read this csv file. It may take about 1min to load the file. 

In [ ]:
df = pd.read_csv('ceph_hgdp_minor_code_XNA.betterAnnotated.csv')
df=df.drop(columns=['snp','chr','pos'])
df=df.T
df


In [ ]:
X=df.to_numpy()
del df # delete the dataframe to free the memory

In [ ]:
print(X.shape) 
#Now we have the data matrix X
print(X)

(1043, 488919)
[[1 2 2 ... 2 1 1]
 [0 1 2 ... 2 1 0]
 [0 2 2 ... 2 1 0]
 ...
 [0 2 2 ... 2 2 2]
 [0 0 1 ... 2 2 2]
 [2 2 2 ... 2 2 1]]


**Step4**: Moreover, the following file contains
the region where each people comes from. 

In [ ]:
!wget https://github.com/yexf308/AdvancedMachineLearning/blob/main/Homework/HW1/ceph_hgdp_minor_code_XNA.sampleInformation.csv?raw=true -O A.csv

--2022-01-21 16:52:28--  https://github.com/yexf308/AdvancedMachineLearning/blob/main/Homework/HW1/ceph_hgdp_minor_code_XNA.sampleInformation.csv?raw=true
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/yexf308/AdvancedMachineLearning/raw/main/Homework/HW1/ceph_hgdp_minor_code_XNA.sampleInformation.csv [following]
--2022-01-21 16:52:28--  https://github.com/yexf308/AdvancedMachineLearning/raw/main/Homework/HW1/ceph_hgdp_minor_code_XNA.sampleInformation.csv
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yexf308/AdvancedMachineLearning/main/Homework/HW1/ceph_hgdp_minor_code_XNA.sampleInformation.csv [following]
--2022-01-21 16:52:28--  https://raw.githubusercontent.com/yexf308/AdvancedMachineLearning/main/Homework/HW1/ceph_hgdp_minor_code_XNA.samp

In [ ]:
df2 = pd.read_csv('A.csv')
df2

,ID,Gender,Population,Geographic.origin,Geographic.area,region,distance,latitude,longtitude
0,HGDP00448,M,Biaka Pygmies,Central African Republic,Central Africa,Africa,2384.859098,4.0,17.0
1,HGDP00479,M,Biaka Pygmies,Central African Republic,Central Africa,Africa,2384.859098,4.0,17.0
2,HGDP00985,M,Biaka Pygmies,Central African Republic,Central Africa,Africa,2384.859098,4.0,17.0
3,HGDP01094,M,Biaka Pygmies,Central African Republic,Central Africa,Africa,2384.859098,4.0,17.0
4,HGDP00982,M,Mbuti Pygmies,Democratic Republic of Congo,Central Africa,Africa,1335.495772,1.0,29.0
...,...,...,...,...,...,...,...,...,...
1038,HGDP00953,M,Yakut,Siberia,Russia,East Asia,9919.809481,63.0,129.5
1039,HGDP01070,F,Sardinian,Italy,Southern Europe,Europe,5305.809561,40.0,9.0
1040,HGDP01376,M,French Basque,France,Southern Europe,Europe,6012.260603,43.0,0.0
1041,HGDP00671,M,Sardinian,Italy,Southern Europe,Europe,5305.809561,40.0,9.0


In [ ]:
label= df2['region'].values # this is the region label for each individual.
print(label.shape)
print(unique(label))

(1043,)
['Africa' 'America' 'Central/South Asia' 'East Asia' 'Europe'
 'Middle East' 'Oceania']


Now you have loaded the data matrix $X$ and the corresponding label. 

# Your project: 
Explore the genetic variation of those persons with their geographic variations, by MDS/PCA.
Since $D$ is big, explore random projections for dimensionality reduction.

Particularly,
projection 1043 persons on the top-2 MDS (PCA) coordinates. Please visualize your results with figures. Make sure to color the data with the region labels. 

In [ ]:
# Your code starts here. 